In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [2]:
gc = GraphCreator("Metrology", include_see_also=False)
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 612 


['Bureau International des Poids et Mesures', 'Metric system', 'Units of measurement', 'French Revolution', 'International System of Units', 'Measurement', 'Conference Generale des Poids et Mesures', 'Metre Convention'] 


['Accuracy and precision', 'Data analysis', 'Dimensional metrology', 'Forensic metrology', 'Geometric dimensioning and tolerancing', 'Historical metrology', 'Instrumentation', 'International vocabulary of metrology', 'Length measurement', 'Metrication', 'Metrologia', 'NCSL International', 'Test method', 'World Metrology Day']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [3]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [4]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [5]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Time translation symmetry,T-symmetry,7
1,T-symmetry,Time translation symmetry,7
2,Tokyo Electron,Nikon,6
3,Metre,Kilogram,6
4,History of measurement,Foot (unit),6


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [6]:
features_df = gc.get_features_df(rank=False)

In [7]:
features_df.sort_values("degree", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link
0,French Revolution,11607,2,10476.0,1131.0,0.002010,1.400952e-02,0.026491,0.210665,2.0,2.0,0.001094,1
1,Coordinated Universal Time,8847,0,8479.0,368.0,0.019351,8.958993e-02,0.012755,0.075676,2.0,1.0,0.018827,0
2,Granite,7142,0,6835.0,307.0,0.000661,1.361544e-03,0.014565,0.044996,1.0,2.0,0.000273,0
3,Academic journal,6477,0,6288.0,189.0,0.001002,9.209188e-04,0.012185,0.018992,1.0,3.0,0.000442,0
4,Gregorian calendar,6210,0,5607.0,603.0,0.027300,9.247140e-02,0.004790,0.114244,3.0,1.0,0.027577,0
5,Islamic calendar,6122,0,4854.0,1268.0,0.029016,8.872291e-02,0.005180,0.237546,3.0,1.0,0.031376,0
6,Year,6019,1,5609.0,410.0,0.027787,1.202433e-01,0.005462,0.057853,2.0,2.0,0.027395,0
7,Climate change,5885,0,5339.0,546.0,0.001783,3.027991e-03,0.012196,0.082980,1.0,2.0,0.000685,0
8,Greenwich Mean Time,5320,0,5033.0,287.0,0.029687,6.296356e-02,0.007396,0.051132,3.0,1.0,0.030140,0
9,Julian calendar,5298,1,4372.0,926.0,0.032051,1.016497e-01,0.003573,0.117458,3.0,2.0,0.035024,0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [8]:
gc.rank_similarity()

gc.features_df.sort_values(["similarity_rank"], ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Metrology,609,1,504.0,105.0,1.000000,8.547434e-02,0.001393,0.009058,0.0,0.0,1.000000,0,2.530462
1,Metre Convention,373,2,49.0,324.0,0.072832,6.598156e-03,0.000331,0.006720,2.0,2.0,0.039474,1,1.230997
2,International Prototype of the Kilogram,135,2,15.0,120.0,0.044248,1.035175e-03,0.000135,0.001461,2.0,2.0,0.023669,0,0.937384
3,Seconds pendulum,274,2,61.0,213.0,0.041078,4.212534e-03,0.000289,0.011980,2.0,2.0,0.016187,0,0.867196
4,Josephson voltage standard,17,2,5.0,12.0,0.006803,4.018110e-05,0.000017,0.000877,3.0,2.0,0.003945,0,0.795735
5,Surface metrology,59,2,19.0,40.0,0.012862,6.803581e-05,0.000032,0.002045,3.0,2.0,0.005769,0,0.780937
6,Pseudoscientific metrology,243,2,176.0,67.0,0.008674,4.762083e-04,0.000209,0.002045,3.0,2.0,0.002950,0,0.710619
7,Water clock,804,2,416.0,388.0,0.166339,8.904110e-02,0.001021,0.057560,3.0,2.0,0.216931,0,0.643931
8,Seismometer,949,2,552.0,397.0,0.011447,4.259537e-04,0.000898,0.084733,2.0,2.0,0.005714,0,0.637404
9,Measurement,1407,1,1120.0,287.0,0.048977,8.416132e-03,0.002497,0.010519,2.0,2.0,0.025900,1,0.529028


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [9]:
evaluate_metrics(gc.features_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.979871,0.999921,0.020050,13.0,0.923077,1.000000,1.000000,1.000000
centrality,0.952690,0.999921,0.047231,13.0,0.230769,1.000000,1.000000,1.000000
adjusted_reciprocity,0.285551,0.999921,0.714370,13.0,0.461538,0.769231,0.846154,0.846154
page_rank,0.955745,0.999921,0.044176,13.0,0.538462,1.000000,1.000000,1.000000
shortest_path_length_from_entry,0.727489,0.999921,0.272432,13.0,0.000000,0.000000,0.000000,0.000000
jaccard_similarity,0.496982,0.999921,0.502939,13.0,0.230769,0.461538,0.615385,0.615385


The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___

In [10]:
df = gc.features_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)
see_also_indeces = []
for node in gc.see_also_articles:
    article = df[df.node == node]
    
    see_also_indeces.append((node, article.index[0]))
    
see_also_indeces

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
len(gc.graph.nodes)